- 概率代理模型：
根据有限的观测值，对函数分布进行估计的工具。
自带某些假设，可以根据少量观测点估计出目标函数的分布（函数值及其置信度）
常见概率代理模型：
高斯过程、高斯混合模型等。
现在最普及的优化库默认基于高斯混合模型的TPE过程

- 采集函数：
确定下一个观测点的规则。
衡量观测点对拟合函数所产生的影响，选取影响最大的点。
常见采集函数：
概率增量PI、期望增量、置信度上界、信息熵等。
现在大部分优化库使用期望增量。

## 导入库，确认使用数据

In [1]:
# 基本工具
import numpy as np
import pandas as pd
import time
import os

# 算法/损失/评估指标等
import sklearn
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.model_selection import KFold,cross_validate

# 优化器
import bayes_opt
from bayes_opt import BayesianOptimization
import hyperopt
from hyperopt import hp,fmin,tpe,Trials,partial
from hyperopt.early_stop import no_progress_loss
import optuna

In [2]:
print(hyperopt.__version__)

0.2.7


In [3]:
print(optuna.__version__)

2.10.0


In [ ]:
data = pd.read_csv(r"/Users/feishuoren/Projects/machine_learning/datasets/HousePrice/train_encode.csv",index_col=0)
data.head()

In [ ]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

X.head()

In [ ]:
X.shape

使用贝叶斯优化是为了提高速度，而高斯过程不够快，因此使用高斯过程的贝叶斯优化逐渐被舍弃

## 基于 Bayes_opt 实现 GP 优化

1.定义目标函数
```
def objective():
    # 评估器
    reg = RFR()
    # 交叉验证
    cv = KFold()
    result = cross_validate(reg,cv)
    # 交叉验证的结果
    loss  = result("test_rmse")
    
    return loss
```

In [ ]:
def bayesopt_objective(n_estimators,max_depth,max_features,min_impurity_decrease):
    # 定义评估器
    # 需要调整的参数等于目标函数的输入，不需要调整的超参数则直接等于固定值
    # 默认参数输入一定是浮点数，因此需要套上int函数整理成整数
    reg = RFR(n_estimators = int(n_estimators)
             ,max_depth = int(max_depth)
             ,max_features = int(max_features)
             ,min_impurity_decrease = min_impurity_decrease
             ,random_state = 1412
             ,verbose = False# 可自行决定是否开启森林建树的verbose
             ,n_jobs = -1)
    
    # 定义损失函数的输出，5折交叉验证下的结果，输出负根均方误差（-RMSE）
    # 注意，交叉验证需要使用数据，但我们不能让数据X,y成为目标函数的输入
    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    validation_loss = cross_validate(reg,X,y
                                  ,scoring = "neg_root_mean_squared_error"
                                  ,cv = cv
                                  ,verbose = False
                                  ,n_jobs = -1
                                  ,error_score = 'raise'
                                  #如果交叉验证中的算法执行报错，则告诉我们错误的理由
                                  )
    
    # 交叉验证输出的评估指标是负根均方误差，因此本来就是负的损失
    # 目标函数可直接输出该损失的均值
    return np.mean(validation_loss["test_score"])                               

2.定义参数空间

In [ ]:
param_grid_simple = {'n_estimators':(80,100)
                    ,'max_depth':(10,25)
                    ,'max_features':(10,20)
                    ,'min_impurity_decrease':(0,1)
                    }

3.定义优化目标函数的具体流程

In [ ]:
def param_bayes_opt(init_points,n_iter):
    # 定义优化器，先实例化优化器
    opt = BayesianOptimization(bayesopt_objective #需要优化的目标函数
                              ,param_grid_simple #备选参数空间
                              ,random_state = 1412 #随机数种子，虽然无法控制住
                              )
    # 使用优化器，记住bayes_opt只支持最大化
    opt.maximize(init_points = init_points # 抽取多少个初始观测值
                ,n_iter = n_iter # 一共观测/迭代多少次
                )
    # 优化完成，取出最佳参数与最佳分数
    params_best = opt.max["params"]
    score_best = opt.max["target"]
    
    # 打印最佳参数与最佳分数
    print("\n","\n","best params:",params_best,
         "\n","\n","best cvscore:",score_best)
    # 返回最佳参数与最佳分数
    return params_best,score_best
                               

4.定义验证函数（非必需）

In [ ]:
def bayes_opt_validation(params_best):
    reg = RFR(n_estimators = int(params_best["n_estimators"])
             ,max_depth = int(params_best["max_depth"])
             ,max_features = int(params_best["max_features"])
             ,min_impurity_decrease = params_best["min_impurity_decrease"]
             ,random_state = 1412
             ,verbose = False
             ,n_jobs = -1)
    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    validation_loss = cross_validate(reg,X,y
                                    ,scoring="neg_root_mean_squared_error"
                                    ,cv=cv
                                    ,verbose=False
                                    ,n_jobs=-1
                                    )
    return np.mean(validation_loss["test_score"])

5.执行实际优化流程

In [ ]:
start = time.time()
params_best,score_best = param_bayes_opt(20,280) # 初始20个观测值，之后迭代280次
print('It takes %s minutes' % ((time.time() - start)/60))
validation_score = bayes_opt_validation(params_best)
print("\n","\n","validation_score:",validation_score)

## 基于Optuna的贝叶斯优化

In [ ]:
optuna.__version__

1. 定义目标函数和参数空间

In [ ]:
def optuna_objective(trial):
    # 定义参数空间
    n_estimators = trial.suggest_int("n_estimators",80,100,1) # 整数型，（参数名称，下界，上界，步长）
    max_depth = trial.suggest_int("max_depth",10,25,1)
    max_features = trial.suggest_int("max_features",10,20,1)
    min_impurity_decrease = trial.suggest_int("min_impurity_decrease",0,5,1)
    
    # 定义评估器
    # 需要优化的参数由上述参数空间决定
    # 不需要优化的参数则直接填写具体值
    reg = RFR(n_estimators = n_estimators
             ,max_depth = max_depth
             ,max_features = max_features
             ,min_impurity_decrease = min_impurity_decrease
             ,random_state = 1412
             ,verbose = False
             ,n_jobs = -1
             )
    # 交叉验证过程，输出负均方根误差（- RMSE）
    # optuna同时支持最大化和最小化，因此如果输出- RMSE，则选择最大化
    # 如果选择输出RMSE，则选择最小化
    cv = KFold(n_splits = 5,shuffle = True,random_state=1412)
    validation_loss = cross_validate(reg,X,y
                                    ,scoring="neg_root_mean_squared_error"
                                    ,cv=cv # 交叉验证模式
                                    ,verbose=False # 是否打印进程
                                    ,n_jobs=-1 # 线程数
                                    ,error_score='raise'
                                    )
    # 最终输出RMSE
    return np.mean(abs(validation_loss["test_score"]))

2.定义优化目标函数的具体流程

In [14]:
def optimizer_optuna(n_trials,algo):
    # 定义使用TPE或者GP
    if algo == "TPE":
        algo = optuna.samplers.TPESampler(n_startup_trials=10,n_ei_candidates=24)

    elif algo == "GP":
        from optuna.integration import SkoptSampler
        import skopt
        algo = SkoptSampler(skopt_kwargs = {'base_estimator':'GP', # 选择高斯过程
                                           'n_initial_points':10, # 初始观测点10个
                                           'acq_func':'EI'} # 选择的采集函数为EI，期望增量
                           )
        
    # 实际优化过程，首先实例化优化器
    study = optuna.create_study(sampler = algo # 要使用的具体算法
                               ,direction = 'minimize' # 优化的方向，可以填写minimize或maxmize
                               )
    # 开始优化，n_trials为允许的最大迭代次数
    # 由于参数空间已经在目标函数汇总定义好，因此不需要输入参数空间
    study.optimize(optuna_objective # 目标函数
                   ,n_trials=n_trials # 最大迭代次数（包括最初的观测值）
                   ,show_progress_bar=True # 要不要展示进度条 
                  )

    # 可直接从优化好的对象study中调用优化的结果
    # 打印最佳参数与最佳损失值
    print("\n","\n","best params:",study.best_trial.params,
         "\n","\n","best score:",study.best_trial.values,
         "\n")

    return study.best_trial.params,study.best_trial.values

3. 执行实际优化流程

In [15]:
import warnings
warnings.filterwarnings('ignore',message='The objective has been evaluated at this point before.')

In [16]:
best_params,best_score = optimizer_optuna(10,"GP") # 默认打印迭代过程

[I 2022-05-12 16:39:39,214] A new study created in memory with name: no-name-5a264b58-c9f1-4845-9dbd-08d8b7dd2171
/Users/feishuoren/anaconda3/lib/python3.8/site-packages/optuna/progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


[I 2022-05-12 16:39:40,587] Trial 0 finished with value: 29488.60038952484 and parameters: {'n_estimators': 92, 'max_depth': 15, 'max_features': 12, 'min_impurity_decrease': 5}. Best is trial 0 with value: 29488.60038952484.
[I 2022-05-12 16:39:42,124] Trial 1 finished with value: 29210.47071276735 and parameters: {'n_estimators': 92, 'max_depth': 16, 'max_features': 18, 'min_impurity_decrease': 2}. Best is trial 1 with value: 29210.47071276735.
[I 2022-05-12 16:39:43,825] Trial 2 finished with value: 28978.47401450302 and parameters: {'n_estimators': 86, 'max_depth': 18, 'max_features': 16, 'min_impurity_decrease': 1}. Best is trial 2 with value: 28978.47401450302.
[I 2022-05-12 16:39:45,183] Trial 3 finished with value: 29548.169540535288 and parameters: {'n_estimators': 84, 'max_depth': 12, 'max_features': 15, 'min_impurity_decrease': 0}. Best is trial 2 with value: 28978.47401450302.
[I 2022-05-12 16:39:46,918] Trial 4 finished with value: 29234.168560230122 and parameters: {'n_est

In [17]:
# optuna.logging.set_verbosity(optuna.logging.ERROR) # 关闭自动打印的info，只显示进度条
optuna.logging.set_verbosity(optuna.logging.INFO)
best_params,best_score = optimizer_optuna(300,"TPE")

[I 2022-05-12 16:39:55,372] A new study created in memory with name: no-name-fe331986-2b3b-41e1-a620-2e86f615cc66
/Users/feishuoren/anaconda3/lib/python3.8/site-packages/optuna/progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


[I 2022-05-12 16:39:56,878] Trial 0 finished with value: 29241.447258310487 and parameters: {'n_estimators': 96, 'max_depth': 19, 'max_features': 13, 'min_impurity_decrease': 2}. Best is trial 0 with value: 29241.447258310487.
[I 2022-05-12 16:39:58,199] Trial 1 finished with value: 29626.373884342098 and parameters: {'n_estimators': 86, 'max_depth': 12, 'max_features': 15, 'min_impurity_decrease': 1}. Best is trial 0 with value: 29241.447258310487.
[I 2022-05-12 16:39:59,439] Trial 2 finished with value: 29651.927712838555 and parameters: {'n_estimators': 83, 'max_depth': 25, 'max_features': 12, 'min_impurity_decrease': 0}. Best is trial 0 with value: 29241.447258310487.
[I 2022-05-12 16:40:00,809] Trial 3 finished with value: 28622.400683646098 and parameters: {'n_estimators': 100, 'max_depth': 24, 'max_features': 14, 'min_impurity_decrease': 5}. Best is trial 3 with value: 28622.400683646098.
[I 2022-05-12 16:40:01,838] Trial 4 finished with value: 29310.484424566886 and parameters:

In [18]:
optuna.logging.set_verbosity(optuna.logging.ERROR)
best_params,best_score = optimizer_optuna(300,"GP")

/Users/feishuoren/anaconda3/lib/python3.8/site-packages/optuna/progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()




 
 best params: {'n_estimators': 89, 'max_depth': 23, 'max_features': 14, 'min_impurity_decrease': 0} 
 
 best score: [28496.546454099334] 



## 基于 HyperOpt 实现TPE优化 
- 目标函数的输入必须是符合hyperopt规定的字典
- Hyoeropt只支持寻找函数f(x)的最小值，不支持寻找最大值

In [19]:
hyperopt.__version__

'0.2.7'

In [20]:
from hyperopt import hp,fmin,tpe,Trials,partial
from hyperopt.early_stop import no_progress_loss

1.定义目标函数

In [21]:
def hyperopt_objective(params):
    # 定义评估器
    # 需要搜索的参数需要从输入的字典中索引出来
    # 不需要搜索的参数，可以是设置好的某个值
    # 在需要整数的参数前调整参数类型
    reg = RFR(n_estimators = int(params["n_estimators"])
             ,max_depth = int(params["max_depth"])
             ,max_features = int(params["max_features"])
             ,min_impurity_decrease = params["min_impurity_decrease"]
             ,random_state=1412
             ,verbose=False
             ,n_jobs=-1)
    
    # 交叉验证结果，输出负根均方误差（- RMSE）
    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    validation_loss = cross_validate(reg,X,y
                                    ,scoring="neg_root_mean_squared_error"
                                    ,cv=cv
                                    ,verbose=False
                                    ,n_jobs=-1
                                    ,error_score='raise'
                                    )
    
    # 最终输出结果，由于只能取最小值，所以必须对（- RMSE）求绝对值
    # 以求解最小RMSE所对应的参数组合
    return np.mean(abs(validation_loss["test_score"]))

2.定义参数空间

In [22]:
param_grid_simple = {'n_estimators':hp.quniform("n_estimators",80,100,1)
                    ,'max_depth':hp.quniform("max_depth",10,25,1)
                    ,"max_features":hp.quniform("max_features",10,20,1)
                    ,"min_impurity_decrease":hp.quniform("min_impurity_decrease",0,5,1)
                    }

In [23]:
# 计算当前参数空间的大小
len([*range(80,100,1)])*len([*range(10,25,1)])*len([*range(10,20,1)])*len([range(0,5,1)])

3000

3.定义优化目标函数的具体流程

In [28]:
def param_hyperopt(max_evals=100):
    # 保存迭代过程
    trials = Trials()
    
    # 设置提前停止
    early_stop_fn = no_progress_loss(100)
    
    # 定义代理模型
    # algo = partial(tpe,suggest,n_startup_jobs=20,n_EI_cadidates=50)
    params_best = fmin(hyperopt_objective # 目标函数
                      ,space = param_grid_simple # 参数空间
                      ,algo = tpe.suggest # 代理模型的选择
                      # ,algo = algo
                      ,max_evals = max_evals # 允许的迭代次数
                      ,verbose = True
                      ,trials = trials
                      ,early_stop_fn = early_stop_fn
                      )
    
    # 打印最优参数，fmin会自动打印最佳分数
    print("\n","\n","best params:",params_best,
         "\n")
    return params_best,trials

4.定义验证函数（非必要）

In [26]:
def hyperopt_validation(params):
    reg = RFR(n_estimators = int(params["n_estimators"])
             ,max_depth = int(params["max_depth"])
             ,max_features = int(params["max_features"])
             ,min_impurity_decrease = params["min_impurity_decrease"]
             ,random_state = 1412
             ,verbose = False
             ,n_jobs=-1
             )
    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    validation_loss = cross_validate(reg,X,y
                                    ,scoring="neg_root_mean_squared_error"
                                    ,cv=cv
                                    ,verbose=False
                                    ,n_jobs=-1
                                    )
    return np.mean(abs(validation_loss["test_score"]))

5.执行实际优化流程

In [29]:
params_best,trials = param_hyperopt(30) # 1%的空间大小

100%|██████████| 30/30 [00:36<00:00,  1.21s/trial, best loss: 28601.70869989323]

 
 best params: {'max_depth': 23.0, 'max_features': 14.0, 'min_impurity_decrease': 2.0, 'n_estimators': 97.0} 



In [30]:
params_best,trials = param_hyperopt(100) # 3%的空间大小

100%|██████████| 100/100 [02:55<00:00,  1.75s/trial, best loss: 28554.697924803077]

 
 best params: {'max_depth': 15.0, 'max_features': 18.0, 'min_impurity_decrease': 2.0, 'n_estimators': 100.0} 



In [31]:
params_best,trials = param_hyperopt(300) # 10%的空间大小

 60%|█████▉    | 179/300 [04:27<03:00,  1.49s/trial, best loss: 28519.483540374506]

 
 best params: {'max_depth': 15.0, 'max_features': 18.0, 'min_impurity_decrease': 0.0, 'n_estimators': 96.0} 



In [32]:
hyperopt_validation(params_best)

28519.483540374506

In [33]:
# 打印所有搜索相关的记录
trials.trials

[{'state': 2,
  'tid': 0,
  'spec': None,
  'result': {'loss': 28787.195919814818, 'status': 'ok'},
  'misc': {'tid': 0,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'max_depth': [0],
    'max_features': [0],
    'min_impurity_decrease': [0],
    'n_estimators': [0]},
   'vals': {'max_depth': [13.0],
    'max_features': [18.0],
    'min_impurity_decrease': [2.0],
    'n_estimators': [87.0]}},
  'exp_key': None,
  'owner': None,
  'version': 0,
  'book_time': datetime.datetime(2022, 5, 12, 11, 59, 56, 728000),
  'refresh_time': datetime.datetime(2022, 5, 12, 11, 59, 58, 550000)},
 {'state': 2,
  'tid': 1,
  'spec': None,
  'result': {'loss': 28772.192469297763, 'status': 'ok'},
  'misc': {'tid': 1,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'max_depth': [1],
    'max_features': [1],
    'min_impurity_decrease': [1],
    'n_estimators': [1]},
   'vals': {'max_depth': [12.0],
    'max_features': [19.0],
    'm